<b>Objetivos</b>
<ol>
    <li>Distribuir 10,000 policías a pie y 1000 patrullas</li>
    <li>Predecir criminalidad: encontrar la colonia con la mayor probabilidad de, en el año 2025, ser la más peligrosa por delitos graves</li>
    <li>Proponer un mecanismo para evaluar la estrategia de distribución de los policías a pie y patrullas</li>
    <li>Recomendar una colonia para instalar una sucursal de una empresa, basándose en los datos</li>
</ol>

<b>Preparación de datos</b>
#Los siguientes pasos se irán borrando de esta celda conforme los completo en el notebook:
    #Separación
        #Conjuntos de entrenamiento, evaluación, prueba (si acaso aplica)
    #Extracción de características
        #Seleccionar características que influyen en el resultado

<p>LIMPIEZA</p>
<li>Información extraviada</li>
<li>Homologar formato</li>
<li>Eliminar ruido: valores alejados, outliers</li>

In [1]:
import pandas as pd
datos = pd.read_csv("../carpetas-de-investigacion-pgj-de-la-ciudad-de-mexico.csv", nrows = 70000000,sep=';')

In [2]:
type(datos)
#Es un DATAFRAME de Pandas

pandas.core.frame.DataFrame

In [3]:
#datos.info()
datos.columns

Index(['ao_hechos', 'mes_hechos', 'fecha_hechos', 'delito', 'categoria_delito',
       'fiscalia', 'agencia', 'unidad_investigacion', 'alcaldia_hechos',
       'colonia_hechos', 'ao_inicio', 'mes_inicio', 'fecha_inicio',
       'calle_hechos', 'calle_hechos2', 'longitud', 'latitud', 'geopoint'],
      dtype='object')

<p>Crear dataframe con las siguientes columnas</p>
<ul>
    <li>fechas_hechos</li>
    <li>delito</li>
    <li>categoria_delito</li>
    <li>alcaldia_hechos</li>
    <li>geopoint</li>
</ul>

In [45]:
#NUEVO DATAFRAME ELIMINANDO COLUMNAS
#newDatos = datos.drop(["ao_hechos", "mes_hechos", "fiscalia", "agencia", "unidad_investigacion", "colonia_hechos", "ao_inicio", "mes_inicio", "fecha_inicio", "calle_hechos", "calle_hechos2"], axis = 1)
#alternatively, I could use inplace = True to alter datos instead of
#reassigning to newDatos

#NUEVO DATAFRAME SELECCIONANDO COLUMNAS
#dropna to eliminate incomplete data
newDatos = datos[["fecha_hechos", "delito", "categoria_delito", "alcaldia_hechos", "geopoint"]].dropna()

In [46]:
newDatos.head()

,fecha_hechos,delito,categoria_delito,alcaldia_hechos,geopoint
0,2016-12-31 22:11:00,HOMICIDIO CULPOSO POR TRÁNSITO VEHICULAR (ATRO...,DELITO DE BAJO IMPACTO,TLAHUAC,"19.3020842299,-99.0535354187"
1,2016-12-31 19:55:00,ROBO DE VEHICULO DE SERVICIO PARTICULAR CON VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,IZTAPALAPA,"19.3630615318,-99.1285499097"
2,2017-01-01 01:15:00,LESIONES INTENCIONALES POR ARMA BLANCA,DELITO DE BAJO IMPACTO,CUAUHTEMOC,"19.4271484491,-99.1254284961"
3,2017-01-01 00:00:00,LESIONES CULPOSAS,DELITO DE BAJO IMPACTO,CUAUHTEMOC,"19.410850532,-99.1442862319"
4,2011-08-17 12:00:00,FRAUDE,DELITO DE BAJO IMPACTO,CUAUHTEMOC,"19.4210705569,-99.1501195725"


In [41]:
#encontrar las diferentes categorias de delito y la cantidad cometida de éstos
newDatos["categoria_delito"].value_counts()

DELITO DE BAJO IMPACTO                                     644562
ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA        51169
ROBO DE VEHÍCULO CON Y SIN VIOLENCIA                        46789
HECHO NO DELICTIVO                                          44378
ROBO A NEGOCIO CON VIOLENCIA                                14386
ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA        9567
ROBO A REPARTIDOR CON Y SIN VIOLENCIA                        8251
HOMICIDIO DOLOSO                                             4230
LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO                3248
ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN VIOLENCIA      3186
VIOLACIÓN                                                    2690
ROBO A CASA HABITACIÓN CON VIOLENCIA                         2053
ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON VIOLENCIA      1616
ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA                 833
ROBO A TRANSPORTISTA CON Y SIN VIOLENCIA                      450
SECUESTRO 

In [57]:
#de las categorías anteriores,
#podemos reagrupar categorías similares...
#para disminuir las categorías.
#newDatos.loc[newDatos["categoria_delito"] == "ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA", newDatos["categoria_delito"]] = "robo_pasajero"
newDatos.loc[newDatos["categoria_delito"] == "ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA"]#, "ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN VIOLENCIA"]

,fecha_hechos,delito,categoria_delito,alcaldia_hechos,geopoint
3335,2017-01-08 07:00:00,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,CUAUHTEMOC,"19.4301305737,-99.160108416"
3431,2017-05-27 02:20:00,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,BENITO JUAREZ,"19.3930918769,-99.1321924339"
3433,2017-05-13 21:00:00,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,GUSTAVO A MADERO,"19.4626171094,-99.0948992051"
5889,2017-06-08 17:30:00,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,GUSTAVO A MADERO,"19.4702916555,-99.0987841828"
7609,2016-11-25 20:00:00,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,MIGUEL HIDALGO,"19.4137979624,-99.1824434992"
7739,2016-12-02 18:00:00,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,CUAUHTEMOC,"19.4111804721,-99.1598489194"
8024,2016-12-05 08:30:00,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,VENUSTIANO CARRANZA,"19.4181076703,-99.1121591267"
8900,2016-12-10 04:00:00,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,MIGUEL HIDALGO,"19.4023837771,-99.1901356528"
8914,2016-11-21 18:30:00,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,IZTAPALAPA,"19.3595576296,-99.096997797"
10711,2016-10-28 12:15:00,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,AZCAPOTZALCO,"19.4964188648,-99.1805806998"


Homologar formato de fecha y reordenar los datos

In [42]:
#Del dataframe, conservar sólo los datos cuya alcaldía está en la CDMX
#Seleccionar alcaldías de la CDMX
alcaldias = [
    "ALVARO OBREGON",
    "AZCAPOTZALCO",
    "BENITO JUAREZ",
    "CUAUHTEMOC",
    "COYOACAN",
    "CUAJIMALPA", "CUAJIMALPA DE MORELOS",
    "GUSTAVO A MADERO",
    "IZTACALCO",
    "IZTAPALAPA",
    "MAGDALENA CONTRERAS", "LA MAGDALENA CONTRERAS",
    "MIGUEL HIDALGO",
    "MILPA ALTA",
    "TLAHUAC",
    "TLALPAN",
    "VENUSTIANO CARRANZA",
    "XOCHIMILCO"
]

datosAlvaro = newDatos[newDatos["alcaldia_hechos"] == alcaldias[0]]
datosAzcapotzalco = newDatos[newDatos["alcaldia_hechos"] == alcaldias[1]]
datosBenito = newDatos[newDatos["alcaldia_hechos"] == alcaldias[2]]
datosCuau = newDatos[newDatos["alcaldia_hechos"] == alcaldias[3]]
datosCoyo = newDatos[newDatos["alcaldia_hechos"] == alcaldias[4]]
datosCuaji = newDatos[(newDatos["alcaldia_hechos"] == alcaldias[5]) | (newDatos["alcaldia_hechos"] == alcaldias[6])]
datosGustavo = newDatos[newDatos["alcaldia_hechos"] == alcaldias[7]]
datosIztacalco = newDatos[newDatos["alcaldia_hechos"] == alcaldias[8]]
datosIztapalapa = newDatos[newDatos["alcaldia_hechos"] == alcaldias[9]]
datosMagdalena = newDatos[(newDatos["alcaldia_hechos"] == alcaldias[10]) | (newDatos["alcaldia_hechos"] == alcaldias[11])]
datosMiguel = newDatos[newDatos["alcaldia_hechos"] == alcaldias[12]]
datosMilpa = newDatos[newDatos["alcaldia_hechos"] == alcaldias[13]]
datosTlahuac = newDatos[newDatos["alcaldia_hechos"] == alcaldias[14]]
datosTlalpan = newDatos[newDatos["alcaldia_hechos"] == alcaldias[15]]
datosVenustiano = newDatos[newDatos["alcaldia_hechos"] == alcaldias[16]]
datosXochi = newDatos[newDatos["alcaldia_hechos"] == alcaldias[17]]

In [43]:
#cuántos delitos de cada tipo hay en esa delegación
datosAlvaro["categoria_delito"].value_counts()

DELITO DE BAJO IMPACTO                                     44224
ROBO DE VEHÍCULO CON Y SIN VIOLENCIA                        2997
HECHO NO DELICTIVO                                          2957
ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA        2640
ROBO A NEGOCIO CON VIOLENCIA                                1112
ROBO A REPARTIDOR CON Y SIN VIOLENCIA                        542
HOMICIDIO DOLOSO                                             326
LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO                225
VIOLACIÓN                                                    211
ROBO A CASA HABITACIÓN CON VIOLENCIA                         171
ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA         93
ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN VIOLENCIA       88
ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON VIOLENCIA       82
ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA                 72
SECUESTRO                                                      6
ROBO A TRANSPORTISTA CON 

In [44]:
#total de delitos en esa delegación
#datosAlvaro["delito"].count()
newDatos.groupby(by = "categoria_delito").count()

,fecha_hechos,delito,alcaldia_hechos,latitud,longitud,geopoint
categoria_delito,,,,,,
DELITO DE BAJO IMPACTO,644562,644562,644562,644562,644562,644562
HECHO NO DELICTIVO,44378,44378,44378,44378,44378,44378
HOMICIDIO DOLOSO,4230,4230,4230,4230,4230,4230
LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO,3248,3248,3248,3248,3248,3248
ROBO A CASA HABITACIÓN CON VIOLENCIA,2053,2053,2053,2053,2053,2053
ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON VIOLENCIA,1616,1616,1616,1616,1616,1616
ROBO A NEGOCIO CON VIOLENCIA,14386,14386,14386,14386,14386,14386
ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN VIOLENCIA,3186,3186,3186,3186,3186,3186
ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,833,833,833,833,833,833


In [26]:
#Delegación con mayor incidencia de delitos
#datosCuau.head(5)

In [27]:
#datosCuau["fecha_hechos"].value_counts()
#01/2019 es un formato distinto

In [ ]:
#Cambiar formato de fecha
datosCuau.loc[datosCuau["Mes y año"] == "01/2019", 'Mes y año'] = "2019-01"

In [ ]:
#dataframe con formato de fecha homologado
datosCuau["Mes y año"].dropna().value_counts()
#datosCuau.dropna()

In [ ]:
#import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
timedf = datosCuau[["Mes y año","Delito","Geopoint"]]
#timedf = timedf.set_index(["Mes y año","Geopoint"])#por qué necesito otra columna? /Geopoint/
timedf.head()
#cuenta = timedf.count(level="Mes y año")#cuántos datos /de cualquier columna/ hay por cada valor de "Mes y año"
#cuenta
#timedf
#timedf.plot()

In [ ]:
#por qué necesito otra columna? /Geopoint, en este caso/
#Si la omito, error: Can only count levels on hierarchical index.
timedf = timedf.set_index(["Mes y año","Geopoint"])
timedf.head()

In [ ]:
cuenta = timedf.count(level="Mes y año")#cuántos datos /de cualquier columna/ hay por cada valor de "Mes y año"
cuenta.head()

In [ ]:
cuenta.plot()

Arriba, gráfica que muestra la cantidad TOTAL de delitos en la alcaldía Cuauhtémoc, mes a mes

Elección del modelo
    #Serie de tiempo ¿AR, ARMA, ARIMA, SARIMA? Elegir de acuerdo con la forma de los datos
    #Agrupamiento por k-means. A partir de qué característica se agruparán.

Entrenamiento y evaluación del modelo
    #Seleccionar parámetros iniciales (p, q, d, etc). Justificar esta selección.
    #Seleccionar número de clusters. Justificar esta selección.
    #Evaluar el modelo (score, accuracy, etc).

<b>Resultados<b>

Observaciones
    #Tendencia secular
    #Variación estacional
    #Variación cíclica
    #Variación irregular
    #¿Se detectó algún patrón?

Inferencia
    #predecir para responder a los objetivos